In [ ]:
import pandas as pd
import hvplot.pandas
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import holoviews as hv
from holoviews import opts, dim
hv.extension('bokeh')

Load precipitation data

In [ ]:
from io import StringIO
import requests
response = requests.get('https://cdec.water.ca.gov/reportapp/javareports?name=DLYPCP')
pcp = pd.read_html(StringIO(response.text))[0]
pcp.columns = pcp.columns.droplevel()
pcp = pcp[pcp.ID.notna()]
pcp = pcp[pcp.ID.str.match('^\\w{3}$')]
pcp['Month to Date'] = pd.to_numeric(pcp['Month to Date'], errors='coerce')
pcp['WY to Date'] = pd.to_numeric(pcp['WY to Date'], errors='coerce')

Load precipitation stations

In [ ]:
response = requests.get('https://cdec.water.ca.gov/reportapp/javareports?name=RealPrecip')
stations = pd.read_html(StringIO(response.text))[0]
stations.columns = stations.columns.droplevel()
stations = stations[stations.ID.str.contains('^\\w{3}$')]
stations
stations.dtypes
stations = stations.astype({'LATITUDE':float,'LONGITUDE':float})
stations = stations[(stations.LATITUDE > 30)]# remove some junk lats

merge on station id

In [ ]:
pcpm = pcp.merge(stations, on='ID')
pcpm = pcpm.dropna(subset='WY to Date')

Map with color and size for precip to date

In [ ]:
map = pcpm.hvplot.points('LONGITUDE','LATITUDE',
                                           geo=True, tiles='CartoLight',#tiles='ESRI',
                                           c='WY to Date', cmap='rainbow4', cnorm='eq_hist',
                                           hover_cols=['Month to Date','WY to Date'],
                                           frame_width=500)#.opts(size=dim('WY to Date'))

map.opts(opts.Points(size=dim('WY to Date')*0.5, alpha=0.66, active_tools=['wheel_zoom']))#, alpha=0.1*dim('Month to Date')))

Delaunay triangulation to a trimesh

In [ ]:
from scipy.spatial import Delaunay
pts = pcpm[['LONGITUDE','LATITUDE']].values
tris = Delaunay(pts)
trimesh = hv.TriMesh((tris.simplices, pcpm[['LONGITUDE','LATITUDE','WY to Date']]))
trimesh = trimesh.opts(
    opts.TriMesh(cmap='rainbow4', edge_color='WY to Date', edge_alpha=1, filled=True, height=400, 
                 #inspection_policy='edges',
                 tools=['hover'], width=400))
trimesh.opts(opts.TriMesh(node_alpha=0, edge_line_alpha=0))

In [ ]:
pts.shape

In [ ]:
tris.nsimplex

In [ ]:
import scipy

In [ ]:
from scipy.interpolate import griddata

In [ ]:
pts.shape

In [ ]:
pcpm['WY to Date'].shape

In [ ]:
import numpy as np

In [ ]:
grid_x, grid_y = np.mgrid[-124:-116:100j, 32:42:200j]

In [ ]:
points = pts
values = pcpm['WY to Date'].values
grid_z0 = griddata(points, values, (grid_x, grid_y), method='nearest')
grid_z1 = griddata(points, values, (grid_x, grid_y), method='linear')
grid_z2 = griddata(points, values, (grid_x, grid_y), method='cubic')

In [ ]:
values

In [ ]:
grid_x.shape, grid_y.shape, grid_z0.shape

In [ ]:
hv.Image((grid_x[0,:], grid_y[:,0], grid_z0)).opts(cmap='rainbow4')